Importing libraries

In [1]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [35]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\data.csv")
data.shape

(160, 2)

Storing url column in data_url

In [36]:
data_url = data['product_url']
data_url

0      https://m.media-amazon.com/images/I/71xWtLhH2G...
1      https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2      https://m.media-amazon.com/images/I/71YG7EClYf...
3      https://m.media-amazon.com/images/I/81ATe15IyH...
4      https://m.media-amazon.com/images/I/61OBcY37KX...
                             ...                        
155    https://m.media-amazon.com/images/I/71U-5XbfSW...
156    https://m.media-amazon.com/images/I/71maWXZscf...
157    https://m.media-amazon.com/images/I/813BY8cbW8...
158    https://m.media-amazon.com/images/I/316YV-DXmF...
159    https://m.media-amazon.com/images/I/81twmbR442...
Name: product_url, Length: 160, dtype: object

Function to check if url contains a image or not

In [37]:
# import requests
# def is_url_image(url):
#    results = []
#    response = requests.head(url, timeout=5)  # Use HEAD request to check headers only
#    content_type = response.headers.get('Content-Type')
#    if content_type and 'image' in content_type:
#       results.append((url, True))
#    else:
#       results.append((url, False))

#    return pd.DataFrame(results, columns=['URL', 'IsImage'])

In [5]:
# import webcolors
# def closest_colour(requested_colour):
#     min_colours = {}
#     for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
#         r_c, g_c, b_c = webcolors.hex_to_rgb(key)
#         rd = (r_c - requested_colour[0]) ** 2
#         gd = (g_c - requested_colour[1]) ** 2
#         bd = (b_c - requested_colour[2]) ** 2
#         min_colours[(rd + gd + bd)] = name
#     return min_colours[min(min_colours.keys())]

# def get_colour_name(requested_colour):
#     try:
#         closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
#     except ValueError:
#         closest_name = closest_colour(requested_colour)
#         actual_name = None
#     return actual_name, closest_name

# requested_colour = (119, 172, 152)
# actual_name, closest_name = get_colour_name(requested_colour)

In [7]:

# def detect_properties_uri(uri):
#     client = vision.ImageAnnotatorClient()
#     image = vision.Image()
#     image.source.image_uri = uri

#     response = client.image_properties(image=image)
#     props = response.image_properties_annotation
#     # print("Properties:",props.dominant_colors.colors)
#     # scr = [col for col in props.dominant_colors.colors]
#     # print(scr)
#     for color in props.dominant_colors.colors:
#         try:
#             r = int(color.color.red)
#             g = int(color.color.green)
#             b = int(color.color.blue)
#             print(f"color: {get_colour_name((r,g,b))}")
#         except Exception as e:
#             print("gone from here",e)
#             continue
        
# detect_properties_uri(data_url[1])

color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'rosybrown')
color: (None, 'whitesmoke')
color: (None, 'darkslategray')
color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'gray')
color: (None, 'black')
color: (None, 'darkslategray')


Empty String variable to store results

In [5]:
txt = ""

Function to extract information from image

In [38]:
def extract_info(uri):
    txt = ''
    l = ''
    try:
        # if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            # res_logo = client.logo_detection(img)
            # logos = res_logo.logo_annotations
            # logo = [log.description for log in logos]
            # txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            # txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)}, "
            l += f"{label}"
            #For retriving Text
            # res_text = client.document_text_detection(img)
            # texts = res_text.text_annotations
            # text = [t.description for t in texts]
            # txt = txt + f"Text: None, " if len(text)==0 else txt + f"Text: {text[0]}, "
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            txt = txt + f"{label}" if len(obj)==0 else txt + f"{obj}"
        
        # else:
        #     txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image"
    return txt,l   

Creating new dataframe to save results

In [39]:
result = data
result.head()

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...


Iterating through function to extract features of image and adding it to result dataframe

In [40]:
for i in range(len(data_url)):
    x=''
    y=''
    x,y = extract_info(data_url[i])
    result.loc[i, ["object", "label"]] = str(x), str(y)  

C:\Users\shrey\AppData\Local\Temp\ipykernel_6064\281301318.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Shoe']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.loc[i, ["object", "label"]] = str(x), str(y)
C:\Users\shrey\AppData\Local\Temp\ipykernel_6064\281301318.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Footwear', 'Shoe', 'Product', 'Walking shoe', 'Grey', 'Sportswear', 'Sneakers', 'Skate shoe', 'Font', 'Outdoor shoe']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.loc[i, ["object", "label"]] = str(x), str(y)


Resultant Dataframe

In [41]:
result

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
155,Skybags Casual Backpack 28L,https://m.media-amazon.com/images/I/71U-5XbfSW...,['Backpack'],"['Light', 'Product', 'Azure', 'Luggage and bag..."
156,Safari Omega spacious/large laptop backpack wi...,https://m.media-amazon.com/images/I/71maWXZscf...,['Backpack'],"['White', 'Light', 'Product', 'Luggage and bag..."
157,BIGBERRY Newage Rodeo 28 Ltrs leatherite/faux ...,https://m.media-amazon.com/images/I/813BY8cbW8...,['Backpack'],"['Horse', 'Grey', 'Material property', 'Electr..."
158,Impulse EmpowerElite 25L Unisex Water Resistan...,https://m.media-amazon.com/images/I/316YV-DXmF...,['Backpack'],"['Luggage and bags', 'Bag', 'Rectangle', 'Comm..."


Saving Dataframe to CSV file

In [42]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)